# Megatron-LM: Training Multi-Billion Parameter Language Models Using Model Parallelism
Focus: ?

References:
- Megatron-LM: Training Multi-Billion Parameter Language Models Using Model Parallelism, 2019 (https://arxiv.org/pdf/1909.08053)

Purpose: very large models can be quite difficult to train due to memory constraints

Approach: implement a simple, efficient intra-layer model parallel approach that enables training transformer models with billions of parameters. The approach does not require a new compiler or library changes, is orthogonal and complimentary to pipeline model paralellism, and can be fully implemented with the insertion of a few communication operations in native PyTorch. (sounds like tensor parallelism)

Results: They converge transformer based models up to 8.3B using 512 GPUs. They sustain 15.1 PetaFLOPs across the entire application with 76% scaling efficiency when compared to a strong single GPU baseline that sustains 39 TeraFLOPs, which is 30% of peak FLOPs. 8.3B GPT-2 like model and 3.9B BERT-like model achieves SOTA on WikiText103 (10.8 vs 15.8) and LAMBADA (66.5 vs 63.2) and RACE (90.9 vs 89.4).

Definitions:
- GEMM = General Matrix-Matrix multiplication aka matmul

Notes:
- Potentially important focus areas:
    - Model parallelism of the attention layer and/or transformer layer (given I already did it for the MLP layer) - Figure 3.
    - Hybrid intra-layer and inter-layer modle parallelism for training a model more than 16B parameters which demands more memory than is available within 16 GPUs of a DGX-2H box
- Data parallelism - weak scaling where incereasing batch size proportionally to number of available workers has linear scaling in training data throughput. But you can't only scale batch size because large batch size training may lead to coplicaitons in the optimization process resulting in reduced accuracy or longer time to convergence, offsetting the benefit of increasing training throughput (ex. too large of a batch size isn't always better).
- Activation (gradient) checkpointing (self.cache): recomputing activations in the backward pass without storing them in the forward pass to reduce memory requirements. Only keep a small subset (checkpoints) one every N layers, then redo the forward computation to regenerate them on the fly. This results in much lower memory and higher compute.
- Core problem to tackle: the model must fit entirely on one worker.
- Parameter sharing = multiple layers reference the same weight tensors in memory. Purpose is to reduce the memory footprint of the model, but this limits the overall capacity of the model.
- Megatron-LM's approach is to utilize model parallelism to split the model across multiple accelerators. This alleviates the memory pressure, and increases the amount of parallelism independently of the microbatch size.

**Model parallelism**

1. Layer-wise pipeline parallelism = groups of operations are performed on one device before the outputs are passed to the next device in the pipeline where a different group of operations are performed. GPipe framework for TensorFlow overcomes the inconsistency issue by using synchronous gradient descent. Main limitations are pipeline bubbles which reduce efficiency, logic to handle efficient pipelining of the communicaiton and computation operations, or changes to the optimizer itself which impact accuracy
2. Distributed tensor computation = orthogonal and more general approach that partitions a tensor operation across multiple devices to accelerate computation or increase model size. Megatron-LM's approach is not to implement a framework and compiler for model parallelism, but a few targeted modifications to existing PyTorch transformer implementations. 
    - The key is that their approach for both layers enables them to perform all GEMMs in a simple transformer layer using only 2 all-reduces in the forward pass and two in the backward pass
    - They also parallelism the embedding layers, E_Hxv, along the vocabulary dimension (column wise). For going from embedding dim H -> vocab dimension v, this requires an all-gather after, and then probably a softmax to obtain probabilities. Because of weight tying, going from vocab dimension v -> embedding dim H uses a row-wise split because it's a E_Hxv^T = E_vxH which means the input embeddings will require an all-reduce otherwise you won't have all hidden dimension contributions to the final hidden embedding dimension. Interesting, the output embedding -> vocab size -> cross entropy loss is actually a fused operation to reduce the dimension from b x s x v -> b x s so you don't need to send over a vocab_size multiplier. (b = batch size, s = sequence length). This just communicates the scalar losses instead of logits, reducing communication and improving the efficiency of model parallelism. Note: this fusing still requires some (b, s) all-reduce operations to calculate things like the sum for the softmax.

<div style="text-align:center;">
    <img src="2025-08-27_Tensor_Parallelism.png" style="width:25%">
</div>

Questions:
- Q1: In the self-attention block for tensor parallelism, is the softmax without considering all QK values fine?

Action items
- Finished reading up to 4. Setup